In [161]:
from datetime import datetime 
import re
import sqlite3
import sys
import numpy as np
import pandas as pd

import messages from textfile. Parse date,author, text (multiline) and message type. The textfile is created by exporting and emailing the chat conversation from the whatsapp chat settings

In [162]:
with open('messages.txt', encoding='utf-8') as file:
    content = file.readlines()
    data = []
    i=0
    while i<len(content):
        message = content[i]
        j=0

        while (i+j+1)<len(content) and not re.match('\d{2}/\d{2}/\d{4}, \d{2}:\d{2} - ',content[i+j+1]):
            messagecont = content[i+j+1]
            #print("adding multiline {}".format(messagecont))
            message = "{} {}".format(message,messagecont)
            i=i+1

        i=i+j

        p = message.find(' - ')
        date = message[0:p-1].strip()
        message = message[p+2:].strip()


        dt = datetime.strptime(date,'%d/%m/%Y, %H:%M')
        p = message.find(':')
        if (p==-1):
            system = True
            author = "system"
            message = message
        else: 
            system = False
            author = message[0:p].strip()
            message = message[p+1:].strip()

        #print("On {} {} wrote {}".format(dt,author,message))
        params = [dt,author,message,system]
        data.append(params)
        #c.execute("INSERT INTO messages (postdate,author,message,system) VALUES (?,?,?,?)",params)


        i=i+1

In [163]:
df = pd.DataFrame(data, columns=['postdate','author','text','systemmessage']  )


# Does Jeff communicate differently with Desi?

posts mentioning desi directly

In [164]:
pd.set_option('display.max_colwidth',120)
pd.set_option('display.max_rows',300)


df[ ( df["author"] == 'Jeff Vb') & ( df["text"].str.lower().str.contains("desi") | (df["text"].str.lower().str.contains("desy"))  ) ]

,postdate,author,text,systemmessage
5350,2016-06-07 09:05:00,Jeff Vb,"Desi, tu paies combien? 😅",False
7997,2017-03-19 15:05:00,Jeff Vb,"Viens au Brésil, Desi, ici on joue tous les jours, et il y a du soleil tout le temps... 😉",False
9525,2017-06-07 11:00:00,Jeff Vb,"Desi, Patrick a insisté pour que je te demande que tu prennes ton bikini mauve, il t'a trouvée admirable dans cet ha...",False
9810,2017-06-17 14:05:00,Jeff Vb,"Desi, à 18:30 avec Heven il nous en manquerait un?",False
10793,2017-07-20 22:03:00,Jeff Vb,"Merci Brian, je viens jouer avec toi Desi, si je peux... 😉",False


what was said when desi is in the conversation? Get the times when desi posted something and get posts by Jeff within some time window

In [166]:
from datetime import datetime, timedelta

windowsizedownmin = 60*10
windowsizeupmin = 60

desiPosts = df[ df["author"] == "Desi Vb" ]
jeffPosts = df[ df["author"] == "Jeff Vb" ]

# which posts of Desi had a response from Jeff within a time window
windowsizemin = 60 
desiposts=desiposts.assign( tmp = np.nan )


conversation = []
for i,post in desiPosts.iterrows():
    addpost = False
    date = post["postdate"]
    end = date + timedelta(minutes=windowsizeupmin)
    start = date - timedelta(minutes=windowsizedownmin)
    
    for j,jpost in jeffPosts.iterrows():
        jdate = jpost["postdate"]
        if start < jdate and jdate < end:
            addpost = True
            conversation.append(jpost)

    if addpost:
            conversation.append(post)

cdf = pd.DataFrame(conversation).drop_duplicates()

cdf.sort_values('postdate')

,postdate,author,text,systemmessage
5065,2016-05-20 10:00:00,Jeff Vb,Moi aussi 17h,False
5069,2016-05-20 10:03:00,Desi Vb,17,False
5073,2016-05-20 17:00:00,Jeff Vb,Moi aussi,False
5074,2016-05-20 17:01:00,Desi Vb,Presque arrivée,False
5078,2016-05-20 19:04:00,Desi Vb,Merde.... j'ai oublié mes lunettes sur le terrain avec le file de ma coop.... il y a quelqu'un que peux le prendre e...,False
5080,2016-05-20 23:01:00,Desi Vb,Merci timo,False
5148,2016-05-25 10:05:00,Jeff Vb,Moi 18h,False
5169,2016-05-25 13:05:00,Jeff Vb,"Tu peux venir faire l'entraînement avec les U15, tu leur prouveras l'efficacité de la défense haute...",False
5171,2016-05-25 14:00:00,Jeff Vb,16,False
5173,2016-05-25 14:01:00,Jeff Vb,yes,False


Who are the 10 most frequent chaters off all times?

In [5]:
df.groupby('author').count().sort_values('text',ascending=False).nlargest(10,'text')

,postdate,text,systemmessage
author,,,
Timo,1288,1288,1288
Leandro Vb,740,740,740
Mirja Vb,658,658,658
system,518,518,518
Junior Vb,412,412,412
Desi Vb,375,375,375
Victor Barbe Vb,298,298,298
Antho Vb,296,296,296
Nico Vb,292,292,292


In [20]:
df=df.assign( tmp = np.nan )
df=df.assign( nrmembers = np.nan )

In [24]:
df.loc[ (df["text"].str.contains("été ajouté") | df["text"].str.contains("Vous avez ajouté") ) & df["systemmessage"] == True , 'tmp' ] = 1
df.loc[ ( df["text"].str.contains("a été retiré") | df["text"].str.contains("est parti") | df["text"].str.contains("Vous avez retiré")  )  & df["systemmessage"] == True , 'tmp' ] = -1


In [31]:
counter=1
for index,row in df[ df["systemmessage"] ].iterrows():
    counter = counter + row["nrmembers"]
    df.loc(index,"nrmembers") = counter
df

SyntaxError: can't assign to function call (<ipython-input-31-e364b506a875>, line 4)

In [25]:
df[ df["systemmessage"] & df["text"].str.contains("Vous avez") ]



,postdate,author,text,systemmessage,tmp,nrmembers
0,2014-08-07 09:02:00,system,"Vous avez changé le sujet en ""beach on rigolait""",True,NaN,NaN
2,2014-08-07 09:02:00,system,Vous avez changé l'icône de ce groupe,True,NaN,NaN
15,2014-08-07 09:05:00,system,"Vous avez changé le sujet en ""beach on rigolai...",True,NaN,NaN
350,2014-08-29 15:04:00,system,Vous avez ajouté Maria Teresa Vb,True,1.0,NaN
511,2014-09-10 09:02:00,system,Vous avez ajouté David geneve volley,True,1.0,NaN
547,2014-09-15 10:03:00,system,Vous avez ajouté Marzia Vb,True,1.0,NaN
609,2014-09-18 12:00:00,system,Vous avez ajouté Natalia Vb,True,1.0,NaN
862,2014-10-04 18:05:00,system,Vous avez ajouté Julie Vb,True,1.0,NaN
863,2014-10-04 19:00:00,system,Vous avez ajouté Paul Vb,True,1.0,NaN
864,2014-10-04 19:00:00,system,Vous avez ajouté Petra Vb,True,1.0,NaN
